# [LongChain Crash Course](https://www.python-engineer.com/posts/langchain-crash-course/)

[https://youtu.be/LbT1yp6quS8](https://youtu.be/LbT1yp6quS8)

LangChain is a framework for developing applications powered by language models. In this LangChain Crash Course you will learn how to build applications powered by large language models. We go over all important features of this framework.

- [GitHub repo](https://github.com/hwchase17/langchain)
- [Official Docs](https://python.langchain.com/en/latest/index.html)

## Overview:

- Installation
- LLMs
- Prompt Templates
- Chains
- Agents and Tools
- Memory
- Document Loaders
- Indexes

Try out all the code in this [Google Colab](https://colab.research.google.com/drive/1VOwJpcZqOXag-ZXi-52ibOx6L5Pw-YJi?usp=sharing).

## Installation

In [ ]:
# !pip install langchain

## LLMs

LangChain provides a generic interface for many different LLMs. Most of them work via their API but you can also run local models.

See all [LLM providers](https://python.langchain.com/en/latest/modules/models/llms/integrations.html).

In [1]:
# pip install openai

# import os
# os.environ["OPENAI_API_KEY"] ="YOUR_OPENAI_TOKEN"

In [1]:
from langchain.llms import OpenAI

llm = OpenAI(temperature=0.9)  # model_name="text-davinci-003"
text = "What would be a good company name for a company that makes colorful socks?"
print(llm(text))



Festive Feet Socks.


In [2]:
# os.environ["HUGGINGFACEHUB_API_TOKEN"] = "YOUR_HUGGINGFACE_TOKEN"

In [3]:
# pip install huggingface_hub

from langchain import HuggingFaceHub

# https://huggingface.co/google/flan-t5-xl
# llm = HuggingFaceHub(repo_id="google/flan-t5-xl", model_kwargs={"temperature":0, "max_length":64})
llm = HuggingFaceHub(repo_id="google/flan-t5-large", model_kwargs={"temperature":0, "max_length":64})

llm("translate English to German: How old are you?")

'Wie alte sind Sie?'

## Prompt Templates

LangChain faciliates prompt management and optimization.

Normally, when you use an LLM in an application, you are not sending user input directly to the LLM. Instead, you need to take the user input and construct a prompt, and only then send that to the LLM.

In [4]:
llm("Can Barack Obama have a conversation with George Washington?")

'no'

A better prompt is this:

In [5]:
prompt = """Question: Can Barack Obama have a conversation with George Washington?

Let's think step by step.

Answer: """
llm(prompt)

'Barack Obama was born in 1961. George Washington died in 1789. So the answer is no.'

This can be achieved with `PromptTemplates`:

In [6]:
from langchain import PromptTemplate

template = """Question: {question}

Let's think step by step.

Answer: """

prompt = PromptTemplate(template=template, input_variables=["question"])

In [7]:
prompt.format(question="Can Barack Obama have a conversation with George Washington?")

"Question: Can Barack Obama have a conversation with George Washington?\n\nLet's think step by step.\n\nAnswer: "

## Chains

Combine LLMs and Prompts in multi-step workflows.

In [8]:
from langchain import LLMChain

llm_chain = LLMChain(prompt=prompt, llm=llm)

question = "Can Barack Obama have a conversation with George Washington?"

print(llm_chain.run(question))

Barack Obama was born in 1961. George Washington died in 1789. So the answer is no.


## Agents and Tools

Agents involve an LLM making decisions about which cctions to take, taking that cction, seeing an observation, and repeating that until done.

When used correctly agents can be extremely powerful. In order to load agents, you should understand the following concepts:

- Tool: A function that performs a specific duty. This can be things like: Google Search, Database lookup, Python REPL, other chains. See available [Tools](https://python.langchain.com/en/latest/modules/agents/tools.html).
- LLM: The language model powering the agent.
- Agent: The agent to use. See also [Agent Types](https://python.langchain.com/en/latest/modules/agents/agents/agent_types.html).

In [9]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent

# pip install wikipedia

from langchain.llms import OpenAI
llm = OpenAI(temperature=0)
tools = load_tools(["wikipedia", "llm-math"], llm=llm)

agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)

In [10]:
agent.run("In what year was the film Departed with Leopnardo Dicaprio released? What is this year raised to the 0.43 power?")



> Entering new AgentExecutor chain...
 I need to find out the year the film was released and then use the calculator to calculate the power.
Action: Wikipedia
Action Input: Departed with Leonardo Dicaprio
Observation: Page: Martin Scorsese and Leonardo DiCaprio
Summary: Martin Scorsese and Leonardo DiCaprio are frequent collaborators in cinema, with DiCaprio appearing in six feature films and one short film made by Scorsese since 2002. The films explore a variety of genres, including historical epic, crime, thriller, biopic, comedy and western. Several have been listed on many critics' year-end top ten and best-of-decade lists.
The duo's films have been nominated for thirty-one Academy Awards, winning nine. In 2013, the duo was awarded National Board of Review Spotlight award for career collaboration. Scorsese's work with DiCaprio is considered to be as vital as his work with Robert De Niro.



Page: Leonardo DiCaprio filmography
Summary: Leonardo DiCaprio is an American actor who be

'The film Departed with Leonardo Dicaprio was released in 2006 and 2006 raised to the 0.43 power is 26.30281917656938.'

## Memory

Add state to Chains and Agents.

Memory is the concept of persisting state between calls of a chain/agent. LangChain provides a standard interface for memory, a collection of memory implementations, and examples of chains/agents that use memory.

In [13]:
from langchain import OpenAI, ConversationChain

llm = OpenAI(temperature=0)
conversation = ConversationChain(llm=llm, verbose=True)

conversation.predict(input="Hi there!")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi there!
AI:

> Finished chain.


" Hi there! It's nice to meet you. How can I help you today?"

In [14]:
conversation.predict(input="Can we talk about AI?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there!
AI:  Hi there! It's nice to meet you. How can I help you today?
Human: Can we talk about AI?
AI:

> Finished chain.


' Absolutely! What would you like to know about AI?'

In [15]:
conversation.predict(input="I'm interested in Reinforcement Learning.")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there!
AI:  Hi there! It's nice to meet you. How can I help you today?
Human: Can we talk about AI?
AI:  Absolutely! What would you like to know about AI?
Human: I'm interested in Reinforcement Learning.
AI:

> Finished chain.


' Sure! Reinforcement Learning is a type of machine learning algorithm that allows an AI agent to learn from its environment by taking actions and receiving rewards or punishments. It is used to solve complex problems that require trial and error. Would you like to know more about how it works?'

## Document Loaders

Combining language models with your own text data is a powerful way to differentiate them. The first step in doing this is to load the data into documents (i.e., some pieces of text). This module is aimed at making this easy.

See all [available Document Loaders](https://python.langchain.com/en/latest/modules/indexes/document_loaders.html).

In [16]:
from langchain.document_loaders import NotionDirectoryLoader

loader = NotionDirectoryLoader("Notion_DB")

docs = loader.load()

## Indexes

Indexes refer to ways to structure documents so that LLMs can best interact with them. This module contains utility functions for working with documents

- Embeddings: An embedding is a numerical representation of a piece of information, for example, text, documents, images, audio, etc.
- Text Splitters: When you want to deal with long pieces of text, it is necessary to split up that text into chunks.
- Vectorstores: Vector databases store and index vector embeddings from NLP models to understand the meaning and context of strings of text, sentences, and whole documents for more accurate and relevant search results. See [available vectorstores](https://python.langchain.com/en/latest/modules/indexes/vectorstores.html).

In [17]:
import requests

url = "https://raw.githubusercontent.com/hwchase17/langchain/master/docs/modules/state_of_the_union.txt"
res = requests.get(url)
with open("state_of_the_union.txt", "w") as f:
  f.write(res.text)

In [18]:
# Document Loader
from langchain.document_loaders import TextLoader
loader = TextLoader('./state_of_the_union.txt')
documents = loader.load()

In [19]:
# Text Splitter
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

In [21]:
# !pip install sentence_transformers

# Embeddings
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings()

In [23]:
# text = "This is a test document."
# query_result = embeddings.embed_query(text)
# query_result

In [24]:
# doc_result = embeddings.embed_documents([text])
# doc_result

In [25]:
# !pip install faiss-cpu

from langchain.vectorstores import FAISS

db = FAISS.from_documents(docs, embeddings)

query = "What did the president say about Ketanji Brown Jackson"
docs = db.similarity_search(query)

In [26]:
print(docs[0].page_content)

Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. 

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.


In [27]:
# Save and load:
db.save_local("faiss_index")
new_db = FAISS.load_local("faiss_index", embeddings)
docs = new_db.similarity_search(query)
print(docs[0].page_content)

Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. 

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.


## End-to-end example¶
Check out the https://github.com/hwchase17/chat-langchain repo.